In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt


In [2]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "dataset",
    image_size=(128, 128),
    batch_size=16
)

class_names = dataset.class_names
print("Classes:", class_names)


Found 549 files belonging to 11 classes.
Classes: ['Apple___Black_rot', 'Corn_(maize)___Common_rust_', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Potato__Early_blight', 'Potato__Healthy', 'Strawberry___Leaf_scorch', 'Tomato___Early_blight', 'Tomato___Healthy', 'Tomato___Late_blight', 'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot']


In [3]:
dataset = dataset.shuffle(1000)

normalization_layer = layers.Rescaling(1./255)
dataset = dataset.map(lambda x, y: (normalization_layer(x), y))


In [4]:
train_size = int(0.8 * len(dataset))
train_data = dataset.take(train_size)
val_data = dataset.skip(train_size)


In [5]:
model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(128,128,3)),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(len(class_names), activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)


C:\Users\SHRUJAN S\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [6]:
history = model.fit(train_data, validation_data=val_data, epochs=8)


Epoch 1/8
28/28 ━━━━━━━━━━━━━━━━━━━━ 8s 173ms/step - accuracy: 0.1693 - loss: 3.0119 - val_accuracy: 0.4554 - val_loss: 1.8989
Epoch 2/8
28/28 ━━━━━━━━━━━━━━━━━━━━ 4s 146ms/step - accuracy: 0.4348 - loss: 1.7054 - val_accuracy: 0.5714 - val_loss: 1.3286
Epoch 3/8
28/28 ━━━━━━━━━━━━━━━━━━━━ 4s 145ms/step - accuracy: 0.6568 - loss: 1.1337 - val_accuracy: 0.7768 - val_loss: 0.6810
Epoch 4/8
28/28 ━━━━━━━━━━━━━━━━━━━━ 4s 145ms/step - accuracy: 0.7735 - loss: 0.7257 - val_accuracy: 0.8020 - val_loss: 0.6383
Epoch 5/8
28/28 ━━━━━━━━━━━━━━━━━━━━ 4s 142ms/step - accuracy: 0.8192 - loss: 0.6117 - val_accuracy: 0.9375 - val_loss: 0.2698
Epoch 6/8
28/28 ━━━━━━━━━━━━━━━━━━━━ 4s 146ms/step - accuracy: 0.9291 - loss: 0.2537 - val_accuracy: 0.9911 - val_loss: 0.1707
Epoch 7/8
28/28 ━━━━━━━━━━━━━━━━━━━━ 4s 140ms/step - accuracy: 0.9817 - loss: 0.1002 - val_accuracy: 1.0000 - val_loss: 0.0579
Epoch 8/8
28/28 ━━━━━━━━━━━━━━━━━━━━ 4s 143ms/step - accuracy: 0.9840 - loss: 0.0857 - val_accuracy: 1.0000 - v

In [7]:
model.save("model/crop_disease_model.h5")
print("Model saved successfully!")


Model saved successfully!


In [1]:
import os
import tensorflow as tf
import numpy as np
import cv2

model = tf.keras.models.load_model("model/crop_disease_model.h5")

class_names = ['Apple___Black_rot',
               'Corn_(maize)___Common_rust_',
               'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',
               'Potato__Early_blight',
               'Potato__Healthy',
               'Strawberry___Leaf_scorch',
               'Tomato___Early_blight',
               'Tomato___Healthy',
               'Tomato___Late_blight',
               'Tomato___Leaf_Mold',
               'Tomato___Septoria_leaf_spot']

test_folder = "test_images"

for filename in os.listdir(test_folder):
    if filename.lower().endswith((".jpg", ".jpeg", ".png")):
        img_path = os.path.join(test_folder, filename)

        img = cv2.imread(img_path)
        img = cv2.resize(img, (128,128))
        img = img / 255.0
        img = np.expand_dims(img, axis=0)

        prediction = model.predict(img)
        class_index = np.argmax(prediction)
        confidence = np.max(prediction) * 100

        print("\nImage:", filename)
        print("Predicted Disease:", class_names[class_index])
        print("Confidence:", round(confidence, 2), "%")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 287ms/step

Image: 00f16858-f392-4d9e-ad9f-efab8049a13f___JR_Sept.L.S 8368.JPG
Predicted Disease: Tomato___Septoria_leaf_spot
Confidence: 66.76 %
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step

Image: 0a08af15-adfe-447c-8ed4-17ed2702d810___RS_L.Scorch 0054.JPG
Predicted Disease: Tomato___Leaf_Mold
Confidence: 98.65 %
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step

Image: 0a41c25a-f9a6-4c34-8e5c-7f89a6ac4c40___FREC_C.Rust 9807.JPG
Predicted Disease: Tomato___Late_blight
Confidence: 43.06 %
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step

Image: 0a5edec2-e297-4a25-86fc-78f03772c100___JR_Sept.L.S 8468.JPG
Predicted Disease: Tomato___Early_blight
Confidence: 42.98 %
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step

Image: 0cd24b0c-0a9d-483f-8734-5c08988e029f___FREC_C.Rust 3762.JPG
Predicted Disease: Tomato___Leaf_Mold
Confidence: 34.73 %
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step

 RS_Rust 1563.JPG
Predicted Disease: Corn_(maize)___Common_rust_
Confidence: 100.0 %


In [1]:
import os
import tensorflow as tf
import numpy as np
import cv2
import pandas as pd

# Load model
model = tf.keras.models.load_model("model/crop_disease_model.h5")

# Disease information dictionary
disease_info = {
    'Apple___Black_rot': ("Fungal disease causing dark lesions.",
                          "Remove infected leaves and apply fungicide."),
    
    'Corn_(maize)___Common_rust_': ("Fungal infection causing reddish pustules.",
                                    "Use resistant varieties and fungicide spray."),
    
    'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)': ("Leaf spot disease in grapes.",
                                                   "Prune affected leaves and apply fungicide."),
    
    'Potato__Early_blight': ("Fungal disease causing brown spots.",
                             "Use crop rotation and fungicide treatment."),
    
    'Potato__Healthy': ("Healthy leaf. No disease detected.",
                        "No action needed."),
    
    'Strawberry___Leaf_scorch': ("Fungal leaf scorch disease.",
                                 "Remove infected leaves and avoid overhead watering."),
    
    'Tomato___Early_blight': ("Common fungal disease causing concentric rings.",
                              "Use fungicide and remove infected leaves."),
    
    'Tomato___Healthy': ("Healthy leaf. No disease detected.",
                         "No action needed."),
    
    'Tomato___Late_blight': ("Serious fungal disease causing rapid leaf decay.",
                             "Apply copper fungicide and ensure proper drainage."),
    
    'Tomato___Leaf_Mold': ("Fungal disease causing yellow patches.",
                           "Improve air circulation and apply fungicide."),
    
    'Tomato___Septoria_leaf_spot': ("Small circular leaf spots with dark borders.",
                                    "Remove infected leaves and use fungicide.")
}

results = []

test_folder = "test_images"

for filename in os.listdir(test_folder):
    if filename.lower().endswith((".jpg", ".jpeg", ".png")):
        img_path = os.path.join(test_folder, filename)

        img = cv2.imread(img_path)
        img = cv2.resize(img, (128,128))
        img = img / 255.0
        img = np.expand_dims(img, axis=0)

        prediction = model.predict(img)
        class_index = np.argmax(prediction)
        confidence = np.max(prediction) * 100
        
        disease_name = list(disease_info.keys())[class_index]
        description, remedy = disease_info[disease_name]

        results.append({
            "Image Name": filename,
            "Predicted Disease": disease_name,
            "Confidence (%)": round(confidence, 2),
            "Description": description,
            "Recommended Remedy": remedy
        })

# Convert to DataFrame
df = pd.DataFrame(results)
df


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 268ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step


,Image Name,Predicted Disease,Confidence (%),Description,Recommended Remedy
0,00c5c908-fc25-4710-a109-db143da23112___RS_Erly...,Tomato___Early_blight,99.639999,Common fungal disease causing concentric rings.,Use fungicide and remove infected leaves.
1,00ce4c63-9913-4b16-898c-29f99acf0dc3___RS_Late...,Tomato___Late_blight,52.980000,Serious fungal disease causing rapid leaf decay.,Apply copper fungicide and ensure proper drain...
2,00e7c4b2-3005-4558-9cfa-235e356cb7a8___RS_Erly...,Tomato___Early_blight,99.620003,Common fungal disease causing concentric rings.,Use fungicide and remove infected leaves.
3,00f16858-f392-4d9e-ad9f-efab8049a13f___JR_Sept...,Tomato___Septoria_leaf_spot,66.760002,Small circular leaf spots with dark borders.,Remove infected leaves and use fungicide.
